# Stochastic volatility
This Notebook demos how to construct a stochastic volatility model and fit it to data. We will use the precision model of G. Chacko and L. M. Viceira. `Dynamic consumption and portfolio choice with stochastic volatility in incomplete markets` given by
\begin{cases}
\mathrm{d}Y_t = \left (\mu + \beta e^{-V_t} \right ) \mathrm{d}t + e^{-V_t/2} \mathrm{d}W_t, \\
\mathrm{d}V_t = \kappa \left (\gamma - V_t \right ) \mathrm{d}t + \sigma \mathrm{d}B_t, \\
\end{cases}
where $\mu, \beta, \gamma \in \mathbb{R}$, and $\kappa, \sigma \in \mathbb{R}_+$. $\{W_t\}$ and $\{V_t\}$ are two one-dimensional, assumed independent, Wiener processes. As the above model is defined in continuous time, we need to discretize it; we do so using the Euler-Maruyma scheme.

We begin with importing the necessary libraries for defining the model.

In [1]:
from pyfilter.timeseries import StateSpaceModel, EulerMaruyma, Observable

Next, we define the governing dynamics. We assume that the initial distribution of the model is given by 
\begin{equation}
V_0 \sim \mathcal{N} \left ( \gamma, \frac{\sigma}{\sqrt{2 \kappa}} \right ),
\end{equation}
so that we get the following functions

In [2]:
import torch

def fh0(reversion, level, std):
    return level


def gh0(reversion, level, std):
    return std / torch.sqrt(2 * reversion)


def fh(x, reversion, level, std):
    return reversion * (level - x)


def gh(x, reversion, level, std):
    return std


def go(vol, level, beta):
    return level # + beta * torch.exp(vol)


def fo(vol, level, beta):
    return torch.exp(-vol / 2)

Next, we shall define our model. In order to do so, we must specify priors for the different parameters. Given their support, we assume that
\begin{cases}
    \mu, \gamma \sim \mathcal{N}(0, 1), \\
    \kappa, \sim \mathcal{E}(5), \\
    \sigma \sim \mathcal{E}(2)
\end{cases}
and for simplicity that $\beta \triangleq 0$. To do this, we need to import the necessary distributions.

In [3]:
from torch.distributions import Exponential, Normal, StudentT, Beta

Next, we define the model in terms of code and get

In [4]:
volparams = Exponential(5.), Normal(0., 1.), Exponential(2.)
logvol = EulerMaruyma((fh0, gh0), (fh, gh), volparams, ndim=1, dt=1.)
obs = Observable((go, fo), (Normal(0., 1.), 0.), StudentT(df=3))

stockmodel = StateSpaceModel(logvol, obs)

And that defines the model. Next, we need a dataset to train on. We're just going to pick one of the stocks represented in the S&P500. We load the data using Quandl and get, starting from 2000

In [5]:
import quandl
import numpy as np

stock = 'HD'
y = np.log(quandl.get('EOD/{:s}'.format(stock), start_date='2000-01-01', column_index=11, api_key='ze2MLfC5LTiv_yqaKx8T', transform='rdiff') + 1)
y *= 100

We plot the data to get an idea of its volatility

In [6]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(16, 9))

y.plot(ax=ax)

In order to fit the model to the given data, we need an algorithm. We shall use a combination of NESS and SMC$^2$. We use SMC$^2$ for the first part of the data set, and then switch to NESS. Furthermore, since we are using particle filters, we need to decide on which proposal to use - should we go with the Bootstrap or something more advanced? For this example, we will use a linearized version. Importing the relevant classes, we get

In [7]:
from pyfilter.algorithms import NESSMC2, NESS
from pyfilter.filters import SISR
from pyfilter.proposals import Linearized

Let's now fit the model to the data

In [8]:
predictions = 10
training = torch.tensor(y.values, dtype=torch.float32)

algs = list()
for i in range(2):
    filt = SISR(stockmodel.copy(), 50, proposal=Linearized())
    
    algs.append(
        NESSMC2(filt, 1000, handshake=1000).initialize().fit(training)
    )

NESS: 100%|████████████████████████████████████████████████████████████████████████| 4787/4787 [37:49<00:00,  3.45it/s]


Next, we plot the posterior distributions

In [ ]:
from pyfilter.utils import normalize
import pandas as pd

fig, ax = plt.subplots(4, figsize=(16, 9))

for r, alg in enumerate(algs):
    param = alg.filter.ssm.observable.theta[0]
    weights = normalize(alg._ness._w_rec)

    xrange, xvals = param.get_plottable(weights=weights, kernel='gaussian')
    ax[0].plot(xrange, xvals, label='Filter {:d}'.format(r+1))

    for i, param in enumerate(alg.filter.ssm.hidden.theta):
        xrange, xvals = param.get_plottable(weights=weights, kernel='gaussian')
        ax[i+1].plot(xrange, xvals, label='Filter {:d}'.format(r+1))

Let's try and predict the future distributions of returns and superimpose the actual returns

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))

actualreturns = y.iloc[-predictions:].cumsum().values
ax.plot(y.index[-predictions:], actualreturns, label='Actual returns')

for i, alg in enumerate(algs):
    px, py = alg.filter.predict(predictions)

    cum_py = np.percentile(np.cumsum(py, axis=0), [1, 50, 99], axis=1).T

    ax.plot(y.index[-predictions:], cum_py, 'r', alpha=0.5)

plt.legend()